# Thử với bỏ qua ràng buộc có thể không dùng hết xe

In [19]:
import numpy as np
import random

In [20]:
num_orders = 5
num_trucks = 2
d = [5, 7, 12, 12, 7]
c = [9, 2, 6, 4, 6]
c1 = [12, 27]
c2 = [27, 31]

## Hàm thích nghi

In [21]:
def fitness(individual):
    quantity = [0] * num_trucks
    cost = [0] * num_trucks
    for i in range(num_orders):
        quantity[individual[i]] += d[i]
        cost[individual[i]] += c[i]

    reward = 0
    penalty = 0

    for i in range(num_trucks):
        if c1[i] <= quantity[i] <= c2[i]:
            reward += quantity[i]
        elif quantity[i] < c1[i]:
            penalty += (c1[i] - quantity[i]) ** 2
        else:
            penalty += (quantity[i] - c2[i]) ** 2 *2
    
    return reward - penalty
        

## Khởi tạo quần thể

In [22]:
def random_generate_populations(pop_size):
    population = []
    for _ in range(pop_size):
        individual = [random.randint(0, num_trucks - 1) for _ in range(num_orders)]
        population.append(individual)
    return population


In [23]:
def greedy_generate_populations(pop_size):
    population = []
    for _ in range(pop_size):
        individual = [-1] * num_orders
        truck_capacity = [0] * num_trucks  # Tổng trọng lượng mỗi xe

        for i in range(num_orders):
            available_trucks = [j for j in range(num_trucks) if truck_capacity[j] + d[i] <= c2[j]]
            if available_trucks:
                chosen_truck = min(available_trucks, key=lambda j: truck_capacity[j])  # Chọn xe có tải nhẹ nhất
                individual[i] = chosen_truck
                truck_capacity[chosen_truck] += d[i]
            else:
                individual[i] = random.randint(0, num_trucks - 1)  # Nếu không có xe hợp lệ, gán ngẫu nhiên

        population.append(individual)
    return population


In [24]:
def hybrid_generation_populations(pop_size,greedy_ratio=0.3):
    greedy_size = int(pop_size * greedy_ratio)
    random_size = pop_size - greedy_size

    population = greedy_generate_populations(greedy_size)
    population = population + random_generate_populations(random_size)

    return population


## Chọn cha mẹ

In [25]:
def random_selection(population, fitnesses):
    return random.choice(population)

In [26]:
def tournament_selection(population, k=5):
    selected = random.sample(population, k)
    selected.sort(key=lambda x: fitness(x), reverse=True)
    return selected[0]

In [27]:
def roulette_wheel_selection(population):
    fitness_values = np.array([fitness(individual) for individual in population])

    # Áp dụng hàm softmax
    exp_fitness = np.exp(fitness_values - np.max(fitness_values))  # Trừ max để tránh tràn số
    probabilities = exp_fitness / np.sum(exp_fitness)
    
    return population[np.random.choice(len(population), p=probabilities)]

In [28]:
def rank_selection(population):
    ranked = sorted(population, key=lambda x: fitness(x), reverse=True)
    ranks = [i+1 for i in range(len(ranked))]
    total_ranks = sum(ranks)
    probabilities = [r/total_ranks for r in ranks]
    return ranked[np.random.choice(len(ranked), p=probabilities)]


## Lai tạo

In [29]:
def one_point_crossover(parent1, parent2):
    point  = random.randint(1, num_orders - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

In [30]:
def two_point_crossover(parent1, parent2):
    point1, point2 = sorted(random.sample(range(len(parent1)), 2))
    child1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    child2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]
    return child1, child2


In [31]:
def uniform_crossover(parent1, parent2):
    child1, child2 = [], []
    for i in range(num_orders):
        if random.random() < 0.5:
            child1.append(parent1[i])
            child2.append(parent2[i])
        else:
            child1.append(parent2[i])
            child2.append(parent1[i])
    return child1, child2

## Đột biến

In [32]:
def random_resetting_mutate(individual, mutate_rate=0.):
    if random.random() < mutate_rate:
        i = random.randint(0, num_orders - 1)
        individual[i] = random.randint(0, num_trucks - 1)
    return individual

In [33]:
def scramble_mutate(individual, mutate_rate=0.1):
    start, end = sorted(random.sample(range(len(individual)), 2))
    subset = individual[start:end]
    random.shuffle(subset)
    individual[start:end] = subset
    return individual

In [34]:
def inversion_mutate(individual, mutate_rate=0.1):
    start, end =sorted(random.sample(range(len(individual)), 2))
    individual[start:end] = individual[start:end][:: -1]
    return individual

In [35]:
def swap_mutate(individual, mutate_rate = 0.1):
    if random.random() < mutate_rate:
        i, j = random.sample(range(num_orders), 2)
        individual[i], individual[j] = individual[j], individual[i]
    return individual

In [36]:
class GA_BPP:
    def __init__(self, fitness=fitness, initialization=hybrid_generation_populations, selection=tournament_selection, crossover=two_point_crossover, mutate=scramble_mutate):
        self.fitness = fitness
        self.initialization = initialization
        self.selection = selection
        self.crossover = crossover
        self.mutate = mutate
    
    def generate_algorithm(self, pop_size=100, num_generations=100,mutate_rate=0.1):
        population = self.initialization(pop_size)
        for _ in range(num_generations):
            next_population = []
            for _ in range(pop_size //2):
                parent1 = self.selection(population)
                parent2 = self.selection(population)
                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutate(child1, mutate_rate)
                child2 = self.mutate(child2, mutate_rate)
                next_population.extend([child1, child2])
            population = next_population + population
            population.sort(key=lambda x: self.fitness(x), reverse=True)
            population = population[:pop_size]

        result = max(population, key=self.fitness)

        return result, self.fitness(result)

In [37]:
model = GA_BPP()
solution, solution_value = model.generate_algorithm()

In [38]:
model_test = GA_BPP(selection=roulette_wheel_selection, crossover=uniform_crossover, mutate=inversion_mutate)
solution_test, solution_value_test = model_test.generate_algorithm()

In [39]:
solution_test = [i+1 for i in solution_test]
print(f'Best solution test: {solution_test}')
print(f'Best solution value test: {solution_value_test}')

Best solution test: [1, 2, 2, 2, 1]
Best solution value test: 43


In [40]:
solution = [i+1 for i in solution]
print(f'Best solution: {solution}')
print(f'Best solution value: {solution_value}')

Best solution: [1, 2, 2, 2, 1]
Best solution value: 43
